In [1]:
from AOC_utils import get_day
import numpy as np

day = 22
input_data = get_day(day)

# print out first few lines to get a feel for the data
input_data[:7]

Day 22 input already downloaded


['6,8,103~7,8,103',
 '1,4,214~1,4,216',
 '1,4,10~1,5,10',
 '4,5,56~4,9,56',
 '7,5,109~8,5,109',
 '9,4,79~9,5,79',
 '1,3,110~1,3,110']

In [2]:
example = '''
1,0,1~1,2,1
0,0,2~2,0,2
0,2,3~2,2,3
0,0,4~0,2,4
2,0,5~2,2,5
0,1,6~2,1,6
1,1,8~1,1,9
'''.split('\n')[1:-1]

In [3]:
len(input_data)

1232

In [94]:
def parse_data(data):
    bricks = [[[int(y) for y in x.split(',')] for x in line.split('~')] for line in data]
    bricks = np.array(bricks)

    grid = np.zeros([bricks[:,1,0].max() + 1, 
                     bricks[:,1,1].max() + 1, 
                     bricks[:,1,2].max() + 1], dtype=int)


    def insert_block(brick, grid):
        grid[brick[0][0]:brick[1][0]+1, brick[0][1]:brick[1][1]+1, brick[0][2]:brick[1][2]+1] += 1
    def remove_block(brick, grid):
        grid[brick[0][0]:brick[1][0]+1, brick[0][1]:brick[1][1]+1, brick[0][2]:brick[1][2]+1] -= 1

    for brick in bricks:
        insert_block(brick, grid)

    # print(grid.sum())
    # let all blocks fall down in the z axis
    
    block_z_min = np.min(bricks[:,:,-1], axis=1)
    lowest_first = np.argsort(block_z_min)
    bricks = bricks[lowest_first]

    # is brick settled
    brick_settled = np.zeros(len(bricks), dtype=bool)

    def is_brick_settled(brick, grid):
        if brick[0][2]-1 < 0:
            return True
        return np.any(grid[brick[0][0]:brick[1][0]+1, brick[0][1]:brick[1][1]+1, brick[0][2]-1] > 0)

    i = 0
    while not np.all(brick_settled):
        # print(i, sum(brick_settled))
        if not brick_settled[i]:
            brick = bricks[i]

            # check underneath the brick
            brick_settled[i] = is_brick_settled(brick, grid)
            if not brick_settled[i]:
                dropdown = 1
                remove_block(brick, grid)
                bricks[i, :, -1] -= dropdown
                brick = bricks[i]
                insert_block(brick, grid)

        i += 1
        i %= len(bricks)
        
    block_z_min = np.min(bricks[:,:,-1], axis=1)
    lowest_first = np.argsort(block_z_min)
    bricks = bricks[lowest_first]

    # check if brick being removed would drop any other bricks
    total = 0
    total2 = 0
    for i in range(len(bricks)):
        brick = bricks[i]
        new_grid = grid.copy()
        remove_block(brick, new_grid)

        can_be_safely_disintegrated = True
        for j in range(len(bricks)):
            if j != i:
                new_brick = bricks[j]
                if not is_brick_settled(new_brick, new_grid):
                    remove_block(new_brick, new_grid)
                    can_be_safely_disintegrated = False
                    total2 += 1
        if can_be_safely_disintegrated:
            total += 1

    print("part 1:", total)
    print("part 2:", total2)

In [95]:
parse_data(input_data)

part 1: 430
part 2: 60558


In [88]:
parse_data(example)

part 1: 5
part 2: 7
